In [5]:
%pip install requests

import requests
import pandas as pd
from datetime import datetime, timedelta

def get_cme_settlements(trade_date: str = None):
    """
    Scrapes the daily settlement data for Singapore FOB Marine Fuel 0.5% from the CME Group website.

    Args:
        trade_date (str, optional): The date for which to fetch data in 'YYYYMMDD' format.
                                    If None, it fetches data for the most recent trading day.
                                    Defaults to None.
    
    Returns:
        pandas.DataFrame: A DataFrame containing the settlement data, or an empty DataFrame if no data is found.
    """
    # If no date is provided, use today's date. The API will automatically find the last available trading day.
    if not trade_date:
        trade_date = datetime.today().strftime('%Y%m%d')

    # This is the "hidden" API endpoint we discovered.
    # The number '4286' is the internal ID for this specific product.
    api_url = f"https://www.cmegroup.com/CmeWS/mvc/Settlements/Future/Settlements/4286/FUT"
    
    # Parameters for the API request. We can get all results by setting a large pageSize.
    params = {
        'tradeDate': trade_date,
        'strategy': 'DEFAULT',
        'pageSize': 500,
    }

    # It's crucial to set a User-Agent header. Many websites block requests that don't look like a real browser.
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    print(f"Fetching data for trade date: {trade_date}...")

    try:
        # Make the GET request to the API
        response = requests.get(api_url, params=params, headers=headers, timeout=10)
        
        # Raise an exception if the request was unsuccessful (e.g., 404 Not Found, 500 Server Error)
        response.raise_for_status()

        # The response is in JSON format, so we parse it into a Python dictionary
        data = response.json()

        # The actual settlement data is in the 'settlements' key
        settlements_data = data.get('settlements', [])

        if not settlements_data:
            print("No settlement data found for this date. It might be a holiday or a future date.")
            return pd.DataFrame()

        # Convert the list of dictionaries into a pandas DataFrame for easy analysis
        df = pd.DataFrame(settlements_data)
        
        print("Successfully retrieved data.")
        return df

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the request: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

# --- Example Usage ---

if __name__ == "__main__":
    # 1. Get the latest available settlement data
    print("--- Getting latest data ---")
    latest_df = get_cme_settlements()
    if not latest_df.empty:
        # We only display a few key columns for readability
        print(latest_df[['month', 'open', 'high', 'low', 'last', 'change', 'settle', 'volume', 'openInterest']])

    print("\n" + "="*50 + "\n")

    # 2. Get data for a specific historical date
    print("--- Getting historical data for May 15, 2024 ---")
    historical_df = get_cme_settlements(trade_date='20240515')
    if not historical_df.empty:
        print(historical_df[['month', 'open', 'high', 'low', 'last', 'change', 'settle', 'volume', 'openInterest']])

   ---------------------------------------- 0.0/64.8 kB ? eta -:--:--
   ---------------------------------------- 64.8/64.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/157.7 kB ? eta -:--:--
   ---------------------------------------- 157.7/157.7 kB 9.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/105.4 kB ? eta -:--:--
   ---------------------------------------- 105.4/105.4 kB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/70.4 kB ? eta -:--:--
   ---------------------------------------- 70.4/70.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/129.8 kB ? eta -:--:--
   ---------------------------------------- 129.8/129.8 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Z.Peng\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


--- Getting latest data ---
Fetching data for trade date: 20250708...
An error occurred during the request: 403 Client Error: Forbidden for url: https://www.cmegroup.com/CmeWS/mvc/Settlements/Future/Settlements/4286/FUT?tradeDate=20250708&strategy=DEFAULT&pageSize=500


--- Getting historical data for May 15, 2024 ---
Fetching data for trade date: 20240515...
An error occurred during the request: 403 Client Error: Forbidden for url: https://www.cmegroup.com/CmeWS/mvc/Settlements/Future/Settlements/4286/FUT?tradeDate=20240515&strategy=DEFAULT&pageSize=500
